# Company register
First attempt: compiling all companies that politicians have declared interests in. Let's start with the swiss parliament.

In [ ]:
import pandas as pd
import numpy as np

SAMPLING = False
SAMPLE_SIZE = 100

In [ ]:
swiss_parliament = pd.read_json("data/parliament_members_interests.json")
# Short sample for testing
if SAMPLING:
    # We don't use pandas sample function because the code breaks
    # if it samples only MPs with no interests.
    swiss_parliament = swiss_parliament[3000:3459]
# Unpack the interests with one row per interest
interests_list = swiss_parliament.groupby(level=0).Interests.apply(
    lambda x: pd.DataFrame(x.values[0])
)

# Join the one-row-per-interest table with the info from the original table
interests_list.index = interests_list.index.droplevel(1)
swiss_parliament = swiss_parliament.drop("Interests", axis=1)

# Replace NaNs with "" so split can work
swiss_parliament_interests = swiss_parliament.join(interests_list, how='inner')[["FirstName", "LastName", "InterestName","FunctionInAgencyText"]].fillna("")
swiss_parliament_interests["InterestName"] = swiss_parliament_interests["InterestName"].apply(lambda x: x.split(",")[0])
swiss_parliament_interests = swiss_parliament_interests.rename(
    columns={
        "FirstName": "first_name",
        "LastName": "last_name",
        "FunctionInAgencyText": "position",
        "InterestName": "interest_name",
    }
)
swiss_parliament_interests["data_source"] = "federal_parliament"

Let's do the same with the Grand Council of Geneva.

In [ ]:
#See the federal parliament import section for detailed explanations.
geneva_parliament = pd.read_json("data/geneva_parliament_members_interests.json")

if SAMPLING:
    geneva_parliament = geneva_parliament[495:499]

interests_list = geneva_parliament.groupby(level=0).interests.apply(
    lambda x: pd.DataFrame(x.values[0])
)

geneva_parliament = geneva_parliament.drop("interests", axis=1)
interests_list.index = interests_list.index.droplevel(1)
geneva_interests = geneva_parliament.join(interests_list, how='inner')[["first_name", "last_name", "company", "position"]].fillna("")
geneva_interests["data_source"] = "geneva_parliament"
geneva_interests = geneva_interests.rename(columns={"company": "interest_name"})

Cool! We now have a list of MPs with their interests, allowing us to lookup an MP from a company name and conversely.

In [ ]:
all_politicians_interests = pd.concat([swiss_parliament_interests, geneva_interests])
all_politicians_interests[all_politicians_interests.interest_name.str.contains("fondation ema", case=False)]#.sample(10)

Now, let's compile a list of unique interests. We will use it later when we will compile a list of board members of those companies.

In [ ]:
all_interests = pd.Series(
    all_politicians_interests.reset_index()["interest_name"].unique()
)
all_interests = pd.DataFrame(
    all_politicians_interests.groupby("interest_name").size().rename('count')
)
all_interests.sort_values(by="count", ascending=False)#.head(10)

In [9]:
all_politicians_interests.reset_index().to_json('data/all_interests.json')